### Install UniversalNER

In [1]:
! git clone https://github.com/universal-ner/universal-ner.git

Cloning into 'universal-ner'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 44 (delta 13), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (44/44), 14.98 KiB | 178.00 KiB/s, done.


In [2]:
%cd universal-ner

/universal-ner


In [3]:
! pip install -r requirements.txt

  Obtaining dependency information for vllm from https://files.pythonhosted.org/packages/e6/de/b4baee3476de5df8e5f779da03811aacd1fdb962c73d9fb5cd0dc09b4bb5/vllm-0.1.4-cp38-cp38-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for transformers>=4.30.1 from https://files.pythonhosted.org/packages/83/8d/f65f8138365462ace54458a9e164f4b28ce1141361970190eef36bdef986/transformers-4.32.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.6 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/df/e7/e0b548208ff5db6323ad974f094e9435adb0a377f35274196fb74adaf58a/gradio-3.41.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 38.2 MB/s eta 0:00:00
  Obtaining dependency information for ray>=2.5.1 from https://files.pythonhosted.org/packages/2a/9c/4ab1fe33db75eab

### Run prediction

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]= "1"

In [3]:
from vllm import LLM
from src.serve.inference import inference
import fire
import ast
import pandas as pd
import numpy as np
from tqdm import tqdm

In [4]:
#models description https://huggingface.co/Universal-NER
llm = LLM(model="Universal-NER/UniNER-7B-type", tensor_parallel_size=1)
# llm = LLM(model="Universal-NER/UniNER-7B-definition", tensor_parallel_size=1)

INFO 08-29 15:52:28 llm_engine.py:70] Initializing an LLM engine with config: model='Universal-NER/UniNER-7B-definition', tokenizer='Universal-NER/UniNER-7B-definition', tokenizer_mode=auto, trust_remote_code=False, dtype=torch.float16, use_dummy_weights=False, download_dir=None, use_np_weights=False, tensor_parallel_size=1, seed=0)


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


INFO 08-29 15:53:03 llm_engine.py:196] # GPU blocks: 1052, # CPU blocks: 512


In [5]:
def predict(text, entity_type,llm):
    examples = [{"conversations": [{"from": "human", "value": f"Text: {text}"}, {"from": "gpt", "value": "I've read this text."}, {"from": "human", "value": f"What describes {entity_type} in the text?"}, {"from": "gpt", "value": "[]"}]}]
    output = inference(llm, examples, max_new_tokens=256)
    if len(ast.literal_eval(output[0])) == 0:
        return np.nan
    else:
        return ast.literal_eval(output[0])[0]

In [6]:
%cd .. 

/


### WDSQ

In [9]:
wdsq = pd.read_csv('workspace/kbqa/kbqa/thesis/ent_linking_res/wdsq_mgenre_with_bela_and_universal_ner.csv')

In [7]:
#check question prediction
cur_ques = "what films have been produced by jun lana?"
predict(cur_ques, 'named entity', llm)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  6.23it/s]


'jun lana'

In [24]:
# predict ner for propmpt: "named entity"
for i in tqdm(range(len(wdsq))):
    try:
        cur_ques = wdsq.loc[i, 'question']
        output = predict(cur_ques, 'named entity', llm)
        wdsq.loc[i, 'universal_ner_def'] = output
    except:
        continue

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 10.53it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 22.30it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.86it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.45it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  5.52it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 13.16it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 10.73it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.45it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.55it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.18it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.39it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.51it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.42it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.55it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 22.25it/s]

Processed 

In [25]:
#check NaN preidctions for propmt:"named entity"
len(wdsq.query('universal_ner_def.isna()'))

99

In [27]:
# predict ner for propmpt: "entity"
for i in tqdm(range(len(wdsq))):
    if pd.isna(wdsq.loc[i, 'universal_ner_def']):
        cur_ques = wdsq.loc[i, 'question']
        output = predict(cur_ques, 'entity', llm)
        wdsq.loc[i, 'universal_ner_def'] = output

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.31it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.36it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 10.34it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  7.58it/s]

100%|██████████| 1382/1382 [00:01<00:00, 1029.56it/s]


In [28]:
#check NaN preidctions for propmt:"entity"

len(wdsq.query('universal_ner_def.isna()'))

1

In [29]:
#separate NER in question with [START] and [END] tokens
for j in tqdm(range(len(wdsq))):
    
    cur_ques = wdsq.loc[j, 'question']
    
    if pd.notna(wdsq.loc[j, 'universal_ner_def']):
        start = cur_ques.find(wdsq.loc[j, 'universal_ner_def'])
        length = len(wdsq.loc[j, 'universal_ner_def'])
        new_ques = wdsq.loc[j, 'question'][:start]+'[START] '+wdsq.loc[j, 'question'][start:start+length]+ ' [END]' + \
        wdsq.loc[j, 'question'][start+length:]
        wdsq.loc[j, 'universal_ner_def_ques'] = new_ques
    else:
        wdsq.loc[j, 'universal_ner_def_ques'] = '[START] '+ wdsq.loc[j, 'question']+' [END]'

100%|██████████| 5622/5622 [00:00<00:00, 5713.85it/s]


In [30]:
wdsq.to_csv('workspace/kbqa/kbqa/thesis/ent_linking_res/wdsq_mgenre_with_bela_and_universal_ner.csv', index=False)

### RuBQ

In [31]:
rubq = pd.read_csv("workspace/kbqa/kbqa/thesis/ent_linking_res/rubq_mgenre_with_bela_and_universal_ner.csv")

In [ ]:
for i in tqdm(range(len(rubq))):
    cur_ques = rubq.loc[i, 'question']
    output = predict(cur_ques, 'named entity', llm)
    rubq.loc[i, 'universal_ner_def'] = output

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 15.25it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 10.66it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 21.37it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.15it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.64it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 23.94it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 18.67it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.73it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 10.83it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.50it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.01it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.68it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 19.63it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.57it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.63it/s]

Processed 

In [55]:
len(rubq.query('universal_ner_upd.isna()'))

50

In [56]:
for i in tqdm(range(len(rubq))):
    if pd.isna(rubq.loc[i, 'universal_ner_upd']):
        cur_ques = rubq.loc[i, 'question']
        output = predict(cur_ques, 'entity', llm)
        rubq.loc[i, 'universal_ner_upd'] = output

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 10.72it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 10.83it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  6.03it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.59it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 24.68it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 24.69it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 24.59it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 13.37it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  8.87it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 24.82it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  8.68it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  7.52it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.85it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 24.55it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 24.98it/s]

Processed 

In [57]:
len(rubq.query('universal_ner_upd.isna()'))

16

In [61]:
for j in tqdm(range(len(rubq))):
    
    cur_ques = rubq.loc[j, 'question']
    
    if pd.notna(rubq.loc[j, 'universal_ner_upd']):
        start = cur_ques.find(rubq.loc[j, 'universal_ner_upd'])
        length = len(rubq.loc[j, 'universal_ner_upd'])
        new_ques = rubq.loc[j, 'question'][:start]+'[START] '+rubq.loc[j, 'question'][start:start+length]+ ' [END]' + \
        rubq.loc[j, 'question'][start+length:]
        rubq.loc[j, 'universal_ner_ques_mgenre_upd'] = new_ques
    else:
        rubq.loc[j, 'universal_ner_ques_mgenre_upd'] = '[START] '+ rubq.loc[j, 'question']+' [END]'

100%|██████████| 1754/1754 [00:00<00:00, 5514.44it/s]


In [62]:
rubq.to_csv('workspace/kbqa/kbqa/thesis/ent_linking_res/rubq_mgenre_with_bela_and_universal_ner.csv', index=False)